In [43]:
pip install scikit-learn

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.0 MB 3.0 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.0 MB 2.8 MB/s eta 0:00:04
   --------- ------------------------------ 2.6/11.0 MB 3.7 MB/s eta 0:00:03
   ---------- ----------------------------- 2.9/11.0 MB 2.9 MB/s eta 0:00:03
   ----------- ---------------------------- 3.1/11.0 MB 3.0 MB/s eta 0:00:03
   -------------- ------------------------- 3.9/11.0 MB 2.8 MB/s eta 0:00:03
   ----------------- ---------------------- 4.7/11.0 MB 2.9 MB/s eta 0:00:03
   --------------------- ------------------ 6.0/11.0 MB 3.3 MB/s eta 0:00:02
   ------------------------- -------------- 7.1/11.0 MB 3.5 MB/s eta 0:00:02
   ----------------------------- ---------- 8.1/11.0 MB 3.6 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.0 MB 3.7 MB/s eta 0:00:01
   ----------

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# 1. Генерация данных
X, y = make_classification(
    n_samples=1000, n_features=20, n_classes=2, n_informative=10, random_state=42
)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Преобразование данных в тензоры
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [4]:
# 2. Определение модели
class SimplePerceptron(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimplePerceptron, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 16),  # Первый слой
            nn.ReLU(),                # Функция активации
            nn.Linear(16, 8),         # Второй слой
            nn.ReLU(),                # Функция активации
            nn.Linear(8, output_dim), # Выходной слой
            nn.Sigmoid()              # Для классификации
        )

    def forward(self, x):
        return self.fc(x)

In [5]:
# 3. Инициализация модели, функции потерь и оптимизатора
model = SimplePerceptron(input_dim=20, output_dim=1)
criterion = nn.BCELoss()  # Бинарная перекрестная энтропия
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [6]:
# 4. Обучение модели
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    
    # Прямой проход
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Обратный проход
    loss.backward()
    optimizer.step()
    
    # Вывод информации об обучении
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [10/50], Loss: 0.6244
Epoch [20/50], Loss: 0.3826
Epoch [30/50], Loss: 0.2098
Epoch [40/50], Loss: 0.1303
Epoch [50/50], Loss: 0.0820


In [7]:
# 5. Оценка модели
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_outputs = (test_outputs > 0.5).float()  # Преобразование в классы (0/1)
    accuracy = (test_outputs == y_test_tensor).sum() / y_test_tensor.size(0)
    print(f"Accuracy on test data: {accuracy:.4f}")

Accuracy on test data: 0.9167


In [8]:
# 6. Сохранение модели
torch.save(model.state_dict(), "simple_perceptron.pth")
print("Model saved as simple_perceptron.pth")

Model saved as simple_perceptron.pth
